In [ ]:
# To generate a PDF out of this, run:
# jupyter nbconvert --to pdf --template notebooks/hidecode notebooks/top5.ipynb

In [ ]:
from pathlib import Path

import pandas as pd
from IPython.display import display, Markdown


raw_data_dir = Path("../../raw_data")
reports_dir = Path("../../reports")
season_id = 25

season_raw_data_dir = raw_data_dir / f"season{season_id}"
season_reports_dir = reports_dir / f"season{season_id}"

pl = pd.read_csv(season_raw_data_dir / "player_logs.csv")

stats_enabled = [
    "pts",
    "fgm",
    "fga",
#     "fg%",
    "3pm",
    "3pa",
#     "3p%",
    "ftm",
    "fta",
#     "ft%",
    "oreb",
    "dreb",
    "reb",
    "ast",
    "stl",
    "tov",
    "blk",
    "blka",
    "fc",
    "fd",
    "pir",
]

stats_groups = [
    ["pir", "Performance Index Rating", ("pts", "reb", "ast", "tov", "stl", "blk", "fc")],
#     ["pts", "Points", ("fgm", "fga", "3pm", "3pa", "ftm", "fta")],
    ["pts", "Points", ("fgm", "fga", "3pm", "3pa", "ftm", "fta")],
    ["reb", "Rebounds", ("oreb", "dreb")],
    ["oreb", "Offensive Rebounds", ("dreb", "reb")],
    ["ast", "Assists", ("tov",)],
    ["stl", "Steals", ("tov", "fc")],
    ["blk", "Blocks", ()],
#     ["fg%", "Field Goal Percentage", ("fgm", "fga")],
#     ["3p%", "Three Point Shot Percentage", ("3pm", "3pa")],
#     ["ft%", "Free Throw Percentage", ("ftm", "fta")],
    ["tov", "Turnovers", ()],
    ["fc", "Fouls Committed", ("fd",)],
#     ["fd", "Fouls Drawn", ()],
]


In [ ]:
def fix_column_names(name):
    if name.endswith("_name"):
        name = name[:-5]
    return name.upper()


player_totals = []
for (team_id, player_id, player_gender), group in pl.groupby(["team_id", "player_id", "player_gender"]):
    pt = {
        "team_id": team_id,
        "team_name": group["team_name"].iloc[0],
        "player_id": player_id,
        "player_gender": player_gender,
        "player_name": group["player_name"].iloc[0],
        **{s: group[s].sum() for s in stats_enabled},
        **{f"{s}_avg": group[s].mean() for s in stats_enabled},
    }
    player_totals.append(pt)

team_totals = []
for team_id, group in pl.groupby("team_id"):
    tt = {
        "team_id": team_id,
        "team_name": group["team_name"].iloc[0],
        **{s: group[s].sum() for s in stats_enabled},
        **{f"{s}_avg": group[s].mean() for s in stats_enabled},
    }
    team_totals.append(tt)
    
ptdf = pd.DataFrame(player_totals)
ttdf = pd.DataFrame(team_totals)

for sort_field, title, other_fields in stats_groups:
    display(Markdown(f"## {title}"))
    
    teams_sorted_by_field = ttdf.sort_values(
        [sort_field, *other_fields],
        ascending=[False, *(False for _ in other_fields)],
    )
    top8_by_field = teams_sorted_by_field[["team_name", sort_field, *other_fields]].head(8)
    top8_by_field.reset_index(drop=True, inplace=True)
    top8_by_field.index += 1
    top8_by_field.columns = map(fix_column_names, top8_by_field.columns)
    display(top8_by_field)

    for gender in ("f", "m"):
        this_gender = ptdf["player_gender"] == gender
        sorted_by_field = ptdf[this_gender].sort_values(
            [sort_field, *other_fields],
            ascending=[False, *(False for _ in other_fields)],
        )
        top5_by_field = sorted_by_field[["player_name", "team_name", sort_field, *other_fields]].head(5)
        top5_by_field.reset_index(drop=True, inplace=True)
        top5_by_field.index += 1
        top5_by_field.columns = map(fix_column_names, top5_by_field.columns)
        display(top5_by_field)
